In [71]:
pd.set_option("display.max_rows",5)
pd.set_option("display.max_columns",41)

In [38]:
import sys
import csv
import operator
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import KFold

In [39]:
data_path = ""
train_file_name = "Train_pjb2QcD.csv"
test_file_name = "Test_wyCirpO.csv"

In [40]:
def getCountVar(compute_df, count_df, var_name, count_var="Manager_Num_Application"):
        grouped_df = count_df.groupby(var_name, as_index=False)[count_var].agg('count')
        grouped_df.columns = [var_name, "var_count"]
        merged_df = pd.merge(compute_df, grouped_df, how="left", on=var_name)
        merged_df.fillna(-1, inplace=True)
        return list(merged_df["var_count"])

In [41]:
def create_feature_map(features):
        outfile = open('xgb.fmap', 'w')
        for i, feat in enumerate(features):
                outfile.write('{0}\t{1}\tq\n'.format(i,feat))
        outfile.close()

In [42]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0):
        params = {}
        params["objective"] = "binary:logistic"
        params['eval_metric'] = 'auc'
        params["eta"] = 0.01 #0.00334
        params["min_child_weight"] = 1
        params["subsample"] = 0.8
        params["colsample_bytree"] = 0.3
        params["silent"] = 1
        params["max_depth"] = 6
        params["seed"] = seed_val
        #params["max_delta_step"] = 2
        #params["gamma"] = 0.5
        num_rounds = 1000 #2500

        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
            xgtest = xgb.DMatrix(test_X, label=test_y)
            watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
            model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=500)
        else:
            xgtest = xgb.DMatrix(test_X)
            model = xgb.train(plst, xgtrain, num_rounds)

        if feature_names:
            create_feature_map(feature_names)
            model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
            importance = model.get_fscore(fmap='xgb.fmap')
            importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
            imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
            imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
            imp_df.to_csv("imp_feat.txt", index=False)

        pred_test_y = model.predict(xgtest)

        if test_y is not None:
            loss = roc_auc_score(test_y, pred_test_y)
            print loss
            return pred_test_y, loss
        else:
            return pred_test_y


In [43]:
print "Reading files.."
train = pd.read_csv(data_path + train_file_name)
test = pd.read_csv(data_path + test_file_name)
print train.shape, test.shape

Reading files..
(9527, 23) (5045, 22)


In [44]:
print "Rank vars.."
prev_date = 0
count_dict = {}
for name, row in train.iterrows():
    count_dict[ row["Application_Receipt_Date"] ] = count_dict.get(row["Application_Receipt_Date"],0) + 1
for name, row in test.iterrows():
    count_dict[ row["Application_Receipt_Date"] ] = count_dict.get(row["Application_Receipt_Date"],0) + 1

Rank vars..


In [45]:
count_dict

{'1/1/2008': 20,
 '1/1/2009': 99,
 '1/10/2008': 30,
 '1/11/2008': 28,
 '1/12/2008': 20,
 '1/14/2008': 9,
 '1/15/2008': 15,
 '1/16/2008': 23,
 '1/17/2008': 25,
 '1/18/2008': 9,
 '1/19/2008': 14,
 '1/2/2008': 85,
 '1/21/2008': 15,
 '1/22/2008': 22,
 '1/23/2008': 19,
 '1/24/2008': 18,
 '1/25/2008': 13,
 '1/26/2008': 9,
 '1/29/2008': 8,
 '1/30/2008': 10,
 '1/31/2008': 14,
 '1/4/2008': 13,
 '1/5/2008': 12,
 '1/7/2008': 25,
 '1/8/2008': 34,
 '1/9/2008': 40,
 '10/1/2007': 6,
 '10/1/2008': 88,
 '10/10/2007': 22,
 '10/10/2008': 28,
 '10/11/2007': 40,
 '10/12/2007': 30,
 '10/13/2007': 27,
 '10/13/2008': 25,
 '10/14/2008': 24,
 '10/15/2007': 19,
 '10/15/2008': 22,
 '10/16/2007': 15,
 '10/16/2008': 30,
 '10/17/2007': 22,
 '10/17/2008': 18,
 '10/18/2007': 46,
 '10/18/2008': 2,
 '10/19/2007': 31,
 '10/2/2007': 31,
 '10/20/2007': 21,
 '10/20/2008': 41,
 '10/21/2008': 19,
 '10/22/2007': 34,
 '10/22/2008': 16,
 '10/23/2007': 28,
 '10/24/2007': 15,
 '10/24/2008': 15,
 '10/25/2007': 1,
 '10/26/2007': 8,


In [46]:
len(count_dict)

500

In [47]:
prev_date = 0
rank_list = []
count_list = []
rankpct_list = []
for name, row in train.iterrows():
    date_value = row["Application_Receipt_Date"]
    if date_value != prev_date:
        rank = 1
        prev_date = date_value
    else:
        rank += 1
    rank_list.append( rank )
    count_list.append( count_dict[date_value] )
    rankpct_list.append( float(rank) / count_dict[date_value] )

In [48]:
len(count_list)

9527

In [49]:
rankpct_list

[0.012658227848101266,
 0.02531645569620253,
 0.0379746835443038,
 0.05063291139240506,
 0.06329113924050633,
 0.0759493670886076,
 0.08860759493670886,
 0.10126582278481013,
 0.11392405063291139,
 0.12658227848101267,
 0.13924050632911392,
 0.1518987341772152,
 0.16455696202531644,
 0.17721518987341772,
 0.189873417721519,
 0.20253164556962025,
 0.21518987341772153,
 0.22784810126582278,
 0.24050632911392406,
 0.25316455696202533,
 0.26582278481012656,
 0.27848101265822783,
 0.2911392405063291,
 0.3037974683544304,
 0.31645569620253167,
 0.3291139240506329,
 0.34177215189873417,
 0.35443037974683544,
 0.3670886075949367,
 0.379746835443038,
 0.3924050632911392,
 0.4050632911392405,
 0.4177215189873418,
 0.43037974683544306,
 0.4430379746835443,
 0.45569620253164556,
 0.46835443037974683,
 0.4810126582278481,
 0.4936708860759494,
 0.5063291139240507,
 0.5189873417721519,
 0.5316455696202531,
 0.5443037974683544,
 0.5569620253164557,
 0.569620253164557,
 0.5822784810126582,
 0.594936708

In [50]:
train.head(20)

,ID,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,...,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,Business_Sourced
0,FIN1000001,842001,4/16/2007,844120.0,M,12/19/1971,M,Others,Graduate,11/10/2005,...,Confirmation,M,2/17/1978,2.0,1.0,335249.0,28.0,335249.0,28.0,0
1,FIN1000002,842001,4/16/2007,844111.0,M,2/17/1983,S,Others,Class XII,11/10/2005,...,Confirmation,M,2/17/1978,2.0,1.0,335249.0,28.0,335249.0,28.0,1
2,FIN1000003,800001,4/16/2007,844101.0,M,1/16/1966,M,Business,Class XII,5/27/2006,...,Confirmation,M,3/3/1969,0.0,0.0,357184.0,24.0,357184.0,24.0,0
3,FIN1000004,814112,4/16/2007,814112.0,M,2/3/1988,S,Salaried,Class XII,8/21/2003,...,Confirmation,F,8/14/1978,0.0,0.0,318356.0,22.0,318356.0,22.0,0
4,FIN1000005,814112,4/16/2007,815351.0,M,7/4/1985,M,Others,Class XII,5/8/2006,...,Confirmation,M,2/7/1971,2.0,1.0,230402.0,17.0,230402.0,17.0,0
5,FIN1000006,814112,4/16/2007,814114.0,M,3/23/1988,S,Others,Class XII,1/17/2006,...,Confirmation,M,2/20/1979,0.0,0.0,247118.0,24.0,247118.0,24.0,1
6,FIN1000007,842001,4/16/2007,844118.0,M,2/5/1969,M,Business,Class XII,9/1/2003,...,Confirmation,M,5/28/1969,0.0,0.0,315119.0,27.0,315119.0,27.0,1
7,FIN1000008,800001,4/16/2007,844103.0,M,1/28/1984,M,Salaried,Class XII,12/16/2006,...,Confirmation,M,1/7/1976,5.0,4.0,117358.0,9.0,117358.0,9.0,0
8,FIN1000009,209625,4/16/2007,206451.0,M,1/8/1976,M,Business,Graduate,11/18/2004,...,Confirmation,M,3/7/1966,0.0,0.0,244028.0,17.0,244028.0,17.0,1
9,FIN1000010,211001,4/16/2007,212218.0,M,2/3/1982,M,Others,Class XII,8/15/2002,...,Confirmation,M,11/14/1974,0.0,0.0,851557.0,39.0,851557.0,39.0,1


In [51]:
train["dayrank"] = rank_list[:]
train["daycount"] = count_list[:]
train["dayrankpct"] = rankpct_list[:]

In [52]:
prev_date = 0
rank_list = []
count_list = []
rankpct_list = []
for name, row in test.iterrows():
    date_value = row["Application_Receipt_Date"]
    if date_value != prev_date:
        rank = 1
        prev_date = date_value
    else:
        rank += 1
    rank_list.append( rank )
    count_list.append( count_dict[date_value] )
    rankpct_list.append( float(rank) / count_dict[date_value] )

In [53]:
test["dayrank"] = rank_list[:]
test["daycount"] = count_list[:]
test["dayrankpct"] = rankpct_list[:]
print train.dayrank.describe(), "\n", test.dayrank.describe()  

count    9527.000000
mean       20.108849
std        18.614587
min         1.000000
25%         7.000000
50%        15.000000
75%        27.000000
max       165.000000
Name: dayrank, dtype: float64 
count    5045.000000
mean       35.843608
std        42.023915
min         1.000000
25%        10.000000
50%        22.000000
75%        44.000000
max       273.000000
Name: dayrank, dtype: float64


In [54]:
print "Getting DV and ID.."
train_y = train.Business_Sourced.values
train_ID = train.ID.values
test_ID = test.ID.values

Getting DV and ID..


In [55]:
print "New feats.."
print "Some more features.."
new_feats = ["DOJ_DOB", "DOB_Applicant_Gender", "DOB_Qualification", "DOB_Gender_Qual"] 
train["DOJ_DOB"] = train["Manager_DOJ"].astype('str') + "_" + train["Manager_DoB"].astype('str')
train["DOB_Applicant_Gender"] = train["Manager_DoB"].astype('str') + "_" + train["Applicant_Gender"].astype('str')
train["DOB_Qualification"] = train["Manager_DoB"].astype('str') + "_" + train["Applicant_Qualification"].astype('str')
train["DOB_Gender_Qual"] = train["Manager_DoB"].astype('str') + "_" + train["Applicant_Gender"].astype('str') + "_" + train["Applicant_Qualification"].astype('str')
test["DOJ_DOB"] = test["Manager_DOJ"].astype('str') + "_" + test["Manager_DoB"].astype('str')
test["DOB_Applicant_Gender"] = test["Manager_DoB"].astype('str') + "_" + test["Applicant_Gender"].astype('str')
test["DOB_Qualification"] = test["Manager_DoB"].astype('str') + "_" + test["Applicant_Qualification"].astype('str')
test["DOB_Gender_Qual"] = test["Manager_DoB"].astype('str') + "_" + test["Applicant_Gender"].astype('str') + "_" + test["Applicant_Qualification"].astype('str')

New feats..
Some more features..


In [56]:
print "Label encoding.."
cat_columns = ["Applicant_Gender", "Applicant_Marital_Status", "Applicant_Occupation", "Applicant_Qualification", "Manager_Joining_Designation", "Manager_Current_Designation", "Manager_Status", "Manager_Gender"]
for f in cat_columns + new_feats:
    print(f), len(np.unique(train[f].values))
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train[f].values) + list(test[f].values))
    train[f] = lbl.transform(list(train[f].values))
    test[f] = lbl.transform(list(test[f].values))
    new_train = pd.concat([ train[['Manager_Num_Application',f]], test[['Manager_Num_Application',f]] ])
    train["CountVar_"+str(f)] = getCountVar(train[['Manager_Num_Application',f]], new_train[['Manager_Num_Application', f]], f)
    test["CountVar_"+str(f)] = getCountVar(test[['Manager_Num_Application',f]], new_train[['Manager_Num_Application',f]], f)

Label encoding..
Applicant_Gender 3
Applicant_Marital_Status 5
Applicant_Occupation 6
Applicant_Qualification 12
Manager_Joining_Designation 9
Manager_Current_Designation 6
Manager_Status 3
Manager_Gender 3
DOJ_DOB 1402
DOB_Applicant_Gender 2030
DOB_Qualification 2396
DOB_Gender_Qual 3287


In [57]:
print "Working on dates.."
for date_col in ["Application_Receipt_Date", "Applicant_BirthDate", "Manager_DOJ", "Manager_DoB"]:
    print date_col
    train[date_col].fillna("1/1/1900", inplace=True)
    test[date_col].fillna("1/1/1900", inplace=True)
    train[date_col] = (pd.to_datetime(train[date_col], format="%m/%d/%Y"))
    test[date_col] = (pd.to_datetime(test[date_col], format="%m/%d/%Y"))
    train[date_col] = train[date_col].apply(lambda x: x.toordinal())
    test[date_col] = test[date_col].apply(lambda x: x.toordinal())

Working on dates..
Application_Receipt_Date
Applicant_BirthDate
Manager_DOJ
Manager_DoB


In [58]:
dev_index = np.where(train["Application_Receipt_Date"]<=733100)[0]
val_index = np.where(train["Application_Receipt_Date"]>733100)[0]
print "Dropping unwanted cols.."
drop_cols = []
train.drop(["ID", "Business_Sourced"]+drop_cols, axis=1, inplace=True)
test.drop(["ID"] + drop_cols, axis=1, inplace=True)

print "Fill NA.."
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)
print "New features.."
train["Manager_Business2"] = train["Manager_Business"] - train["Manager_Business2"]
test["Manager_Business2"] = test["Manager_Business"] - test["Manager_Business2"]
train["Manager_Num_Products2"] = train["Manager_Num_Products"] - train["Manager_Num_Products2"]
test["Manager_Num_Products2"] = test["Manager_Num_Products"] - test["Manager_Num_Products2"]

Dropping unwanted cols..
Fill NA..
New features..


In [72]:
train

,Office_PIN,Application_Receipt_Date,Applicant_City_PIN,Applicant_Gender,Applicant_BirthDate,Applicant_Marital_Status,Applicant_Occupation,Applicant_Qualification,Manager_DOJ,Manager_Joining_Designation,Manager_Current_Designation,Manager_Grade,Manager_Status,Manager_Gender,Manager_DoB,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,dayrank,daycount,dayrankpct,DOJ_DOB,DOB_Applicant_Gender,DOB_Qualification,DOB_Gender_Qual,CountVar_Applicant_Gender,CountVar_Applicant_Marital_Status,CountVar_Applicant_Occupation,CountVar_Applicant_Qualification,CountVar_Manager_Joining_Designation,CountVar_Manager_Current_Designation,CountVar_Manager_Status,CountVar_Manager_Gender,CountVar_DOJ_DOB,CountVar_DOB_Applicant_Gender,CountVar_DOB_Qualification,CountVar_DOB_Gender_Qual
0,842001,732782,844120.0,1,719880,1,1,8,732260,0,1,3.0,0,1,722132,2.0,1.0,335249.0,28.0,0.0,0.0,1,79,0.012658,297,853,1018,1419,9913,9120,2224,5840,5849,4648,7206,11321,17,16,6,5
1,842001,732782,844111.0,1,723958,2,1,7,732260,0,1,3.0,0,1,722132,2.0,1.0,335249.0,28.0,0.0,0.0,2,79,0.025316,297,853,1017,1418,9913,3847,2224,6527,5849,4648,7206,11321,17,16,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9525,575003,733224,571248.0,1,725363,2,2,7,733198,2,2,4.0,1,1,721620,0.0,0.0,0.0,0.0,0.0,0.0,17,18,0.944444,1200,2550,3066,4264,9913,3847,5702,6527,2483,3563,5859,11321,19,11,7,4
9526,411006,733224,411006.0,0,721391,1,1,8,693596,8,5,-999.0,2,2,693596,-999.0,-999.0,-999.0,-999.0,0.0,0.0,18,18,1.000000,1833,2633,3175,4407,3093,9120,2224,5840,0,0,0,0,0,0,0,0


In [33]:
print "Converting to array.."
feat_names = list(train.columns)
train = np.array(train)
test = np.array(test)
print train.shape, test.shape
assert train.shape[1] == test.shape[1]

Converting to array..
(9527, 40) (5045, 40)


In [37]:
full_preds = 0
for rs in [1, 1343, 445234]:
    preds = runXGB(train, train_y, test, feature_names=feat_names, seed_val = rs)
    full_preds += preds

full_preds /= 3.
    
out_df = pd.DataFrame({"ID":test_ID})
out_df["Business_Sourced"] = full_preds
out_df.to_csv("final.csv", index=False)